In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
import datetime
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

In [5]:
from pathlib import Path
import torch
import sys
SCRIPT_DIR = Path('/home/alina.smolina/DL-pipeline/src')
sys.path.append(str(SCRIPT_DIR))

from module import SegModule
model_inference = SegModule.load_from_checkpoint(
    # "/beegfs/home/alina.smolina/DL-pipeline/weights/group-2classes-sakhalin/lucky-mountain-3/epoch=169-val_loss=0.0000.ckpt",
    "/beegfs/home/alina.smolina/DL-pipeline/weights/group-2groups-sakhalin/glamorous-spaceship-8/epoch=499-val_loss=0.00000.ckpt",
)

In [6]:
from datamodule_eolearn import GeoEOModule

import torch
torch.set_float32_matmul_precision('high')

In [7]:
import albumentations as A
import albumentations.pytorch as AP

transforms = A.Compose([
    A.Flip(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=(-0.0625, 0.0625), 
        scale_limit=0, #no scale
        rotate_limit=(-90, 90), 
        p=0.5
    ),
    AP.ToTensorV2(transpose_mask=True),
    ],
)

test_transform = A.Compose([
    AP.ToTensorV2(transpose_mask=True),
    ],
)

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score
import pytorch_lightning as pl
from torchmetrics.classification import F1

class SegmentationModel(pl.LightningModule):
    def __init__(self, model):
        super(SegmentationModel, self).__init__()
        self.model = model
        self.f1_metric = F1(num_classes=2)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # Your training step logic here
        pass

    def validation_step(self, batch, batch_idx):
        # Your validation step logic here
        pass

    def configure_optimizers(self):
        # Define your optimizer and scheduler here
        pass

    def threshold_search(self, val_loader):
        # Set your model to evaluation mode
        self.model.eval()

        best_threshold = 0.0
        best_f1 = 0.0

        with torch.no_grad():
            for data in val_loader:
                inputs, targets = data
                outputs = self.model(inputs)
                probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                targets = targets.cpu().numpy()

                # Iterate over thresholds and find the one with the highest F1 score
                for threshold in range(0, 101):
                    threshold /= 100.0
                    preds = (probs > threshold).astype(int)
                    f1 = f1_score(targets, preds)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_threshold = threshold

        print(f"Best F1: {best_f1}, Best Threshold: {best_threshold}")
        return best_threshold

# Example usage
model = YourSegmentationModel()  # Replace with your segmentation model
segmentation_model = SegmentationModel(model)

# Assuming you have a validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
best_threshold = segmentation_model.threshold_search(val_loader)
